<a href="https://colab.research.google.com/github/akiragm/DeepLearnng/blob/main/NLP_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sentencepiece as spm

# SentencePieceでのトークナイゼーション

In [17]:
# データの読み込み
data = pd.read_csv('/content/AI_traning_data.csv')

# ここでは'data'がテキストカラムだと仮定します。適切なカラム名に置き換えてください。
texts = data.values

# データを一時的なファイルに書き出し
with open('/content/texts.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text + '\n')

spm.SentencePieceTrainer.train('--input=/content/texts.txt --model_prefix=sp --vocab_size=2000')
sp = spm.SentencePieceProcessor()
sp.load('sp.model')


UnicodeDecodeError: ignored

In [ ]:
tokenized_texts = [sp.encode_as_ids(text) for text in texts]

In [ ]:
# ハイパーパラメータ
MAX_LEN = 100
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# パディング
padded_texts = tf.keras.preprocessing.sequence.pad_sequences(tokenized_texts, maxlen=MAX_LEN, padding='post')

# データセットの作成
dataset = tf.data.Dataset.from_tensor_slices(padded_texts)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


# Transformer Model

In [ ]:
def build_transformer_model(vocab_size=2000, embed_dim=256, num_heads=4, feed_forward_dim=512, num_layers=4, max_len=100):
    inputs = keras.Input(shape=(max_len,))
    embedding_layer = TokenEmbedding(vocab_size, max_len, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerEncoderStack(embed_dim, num_heads, feed_forward_dim, num_layers)
    x = transformer_block(x, training=True)  # training=TrueでDropoutやNormalizationを有効にする
    # ここでは、最後のトークンをターゲットとして予測するためのシンプルな出力層を追加します。
    outputs = layers.Dense(vocab_size, activation="softmax")(x[:, -1, :])
    return keras.Model(inputs=inputs, outputs=outputs)

model = build_transformer_model()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


# ModelFit

In [ ]:
# 学習データと検証データに分割
train_size = int(0.9 * len(padded_texts))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

model.fit(train_dataset, validation_data=val_dataset, epochs=5)
